In [ ]:
from importlib import reload
import ipynb.fs.full.notebook_viewer
reload(ipynb.fs.full.notebook_viewer)
from ipynb.fs.full.notebook_viewer import Viewer, root, bucket, fname2id, id2fname, split_dump, Filter
import pickle
from pickle import Unpickler
from pprint import pprint
from dotdict import dotdict
from functools import reduce

from typing import *
import matplotlib.pyplot as plt
import random
import PIL
from pathlib import Path
import json
from collections import defaultdict
from vivid.utils import timer
import subprocess
from subprocess import PIPE
from tqdm.notebook import tqdm
from time import time
from functools import lru_cache
import numpy as np
from PIL import Image
import imagesize
unit = 1
cmap = plt.get_cmap("tab10")
cmap

In [ ]:
master = {
    "v3-mirror-auto4-noaug":{
        "paths":[root/"training116/00013-v3-mirror-auto4-noaug"],
        "color": cmap(0),
        "label": "背景白で腰上の立ち絵,2万枚,augなし",
    },
    "v4-mirror-auto4-noaug":{
        "paths":[root/"training116/00015-v4-mirror-auto4-noaug"],
        "color": cmap(4),
        "label": "背景白で腰上の立ち絵,8万枚,augなし",
    },
    "v3-mirror-auto4-bg":{
        "paths": [root/"training115/00009-v3-mirror-auto4-bg", root/"training115/00010-v3-mirror-auto4-bg-resumecustom", root/"training116/00014-v3-mirror-auto4-bg-resumecustom"],
        "color": cmap(1),
        "label": "背景白で腰上の立ち絵,2万枚,augはbitblit+geometry",
    },
    "portraits-mirror-auto4-noaug":{
        "paths":[root/"training116/00009-portraits-mirror-auto4-noaug"],
        "color": cmap(2),
        "label": "顔アップ,30万枚,augなし",
    },
    "portraits-mirror-auto4":{
        "paths":[root/"training116/00010-portraits-mirror-auto4"],
        "color": cmap(3),
        "label": "顔アップ,30万枚,augはbitblit+geometry+color",
    },
}

In [ ]:
class Result(NamedTuple):
    path: Path
    metric: Dict[str, Any]

class Results:
    def __init__(self, paths):
        self.results = list()
        for i, path in enumerate(paths):
            for j, line in enumerate((path/"metric-fid50k_full.jsonl").read_text().strip().split("\n")):
                if i != 0 and j == 0: continue
                metric = json.loads(line)
                self.results.append(Result(path, metric))
    def __len__(self):
        return len(self.results)
    def fid50k_full(self):
        return [metric["results"]["fid50k_full"] for path, metric in self.results]
    def image(self, i):
        path, metric = self.results[i]
        num = metric["snapshot_pkl"].split("-")[-1].split(".")[0]
        return path/f"fakes{num}.png"

In [ ]:
def myplot(names: List[str], best=1):
    fig = plt.figure(facecolor="white", dpi=100)
    ax = fig.add_subplot()
#    plt.rcParams["font.size"] = 12.5
    plt.rcParams["font.family"] = "IPAPGothic"
    plt.grid()
    for name in names:
        results = Results(master[name]["paths"])
        Y = results.fid50k_full()
        X = [unit*i for i in range(len(Y))]
        YX = list(zip(Y, X))
        YX.sort()
        bests = set()
        for i in range(best):
            y, x = YX[i]
            bests.add(y)
#            ax.text(x, y-50, f"{y:.1f}\n{x}\n{results.image(x)}", color=master[name]["color"])
            ax.text(x, y-50, f"{y:.1f}\n{x}", color=master[name]["color"])
            plt.scatter([x], [y], marker="x", color=master[name]["color"])
        plt.plot(X, Y, label=master[name].get("label", name), marker="|", color=master[name]["color"])

    plt.xlabel("*200kimg")
    plt.ylabel("fid50k_full")
    plt.legend()

In [ ]:
myplot(["v3-mirror-auto4-bg","v3-mirror-auto4-noaug", "v4-mirror-auto4-noaug"])